#**Convolution: 합성곱**

1. 두 배열 x와 w가 있다고 가정하자.   
    x = [2, 8, 3, 7, 1, 2, 0, 4, 5]   
    w = [2, 1, 5, 3]

2. 여기서 원소수가 적은 배열 w를 뒤집어 배열 x의 왼쪽 끝자리에 맞춘다.   
    x = [2, 8, 3, 7, 1, 2, 0, 4, 5]   
    w_r = [3, 5, 1, 2]

3. 그 다음 각 배열의 원소끼리 곱한 후 더한다.(점 곱 연산)   
  $2 × 3 + 8 × 5 + 3 × 1 + 7 × 2 = 63$

4. 이제는 w_r을 오른쪽으로 한 칸 이동하여 배열끼리 곱한 후 더한다.   
  $8 × 3 + 3 × 5 + 7 × 1 + 1 × 2 = 48$

5. 위 과정을 배열 x의 끝에 도착할 때까지 반복한다.

>합성곱으로 얻어진 값은 각각 63, 48, 49, 28, 21, 20 이다.

합성곱은 $x * w$로 표기한다.

합성곱을 넘파이로 구현해보자.

In [1]:
import numpy as np
w = np.array([2,1,5,3])
x = np.array([2,8,3,7,1,2,0,4,5])

In [2]:
w_r = np.flip(w)
# w_r = w[::-1]
print(w_r)

[3 5 1 2]


In [3]:
for i in range(6):
  print(np.dot(x[i:i+4], w_r))

63
48
49
28
21
20


###**싸이파이로 합성곱 수행하기**

싸이파이는 합성곱을 위한 함수 convolve()를 제공한다.

In [4]:
from scipy.signal import convolve
convolve(x, w, mode='valid')

array([63, 48, 49, 28, 21, 20])

#**Cross-Correlation: 교차 상관**

사실 합성곱 신경망은 합성곱을 사용하지 않는다.   
대부분의 딥러닝 패키지들은 합성곱 대신 '교차상관'연산을 사용한다.

교차상관은 합성곱과 동일한 방법으로 연산이 진행되지만,   
'미끄러지는 배열을 뒤집지 않는다'는 점이 다르다.

x = [2, 8, 3, 7, 1, 2, 0, 4, 5]   
w = [2, 1, 5, 3]   
$2 × 2 + 8 × 1 + 3 × 5 + 7 × 3 = 48$

교차상관은 correlate() 메서드를 사용해 계산할 수 있다.

In [5]:
from scipy.signal import correlate
correlate(x, w, mode='valid')

array([48, 57, 24, 25, 16, 39])

###**합성곱 신경망에서 왜 교차상관을 사용할까**

모델을 훈련할 때 가중치를 무작위로 초기화한다.   
앞에서 합성곱을 설명하면서 예시로 든 **'미끄러지는 배열'**이 **가중치**에 해당한다.   
따라서 가중치는 무작위로 초기화하기 때문에 **순서가 뒤집히던 뒤집히지 않던 상관이 없다**.   
하지만 이미 '합성곱' 신경망이라는 이름이 관례적으로 널리 사용되어 있으므로 개념과 용어를 혿동하지 말자.

#**Padding: 패딩**

패딩은 원본 배열의 양 끝에 **빈 원소를 추가**하는 것이다.   

우리가 앞에서 사이파이로 구현할 때 mode='valid'를 지정했다.   
이것이 **'밸리드 패딩'**이다.   

##**Valid Padding: 밸리드 패딩**
밸리드 패딩은 원본 배열에 패딩을 추가하지 않고   
미끄러지는 배열이 원본 배열의 끝으로 갈 때까지 교차상관을 수행한다.   
이로 인해 밸리드 패딩의 결과로 얻은 배열의 크기는 원본 배열보다 항상 작다.   
밸리드 패딩의 특징은 원본 배열의 각 원소가 연산에 참여하는 정도가 다르다는 것이다.

    x = [2, 8, 3, 7, 1, 2, 0, 4, 5]   
    w = [2, 1, 5, 3]   
       w = [2, 1, 5, 3]   
          w = [2, 1, 5, 3]   
             w = [2, 1, 5, 3]   

위 그림을 보면 원본 배열의 첫 번째 원소(2)는 연산에 1번만 참여한다는 것을 알 수 있다.   
하지만 네 번째 원소(7)는 4번의 연산에 참여한다.

즉, 밸리드 패딩의 경우 원본 배열의 양 끝 원소의 연산 참여도가 낮다.




원본 배열의 원소가 연상에 동일하게 참여하게 하려면 원본 배열의 양 끝에 가상의 원소를 추가해야 한다.   
이때 가상의 원소를 0으로 놓기 때문에 이를 **'제로패딩(zero padding)'**이라고 한다.

#**Full Padding: 풀 패딩**

원본 배열의 양 끝에 가상의 원소를 추가하여 모든 원소가 동일하게 연산에 참여하도록 하는 패딩 방식

    x = [0, 0, 0, 2, 8, 3, 7, 1, 2, 0, 4, 5, 0, 0, 0]   
    w = [2, 1, 5, 3]   
       w = [2, 1, 5, 3]   
          w = [2, 1, 5, 3]   
             w = [2, 1, 5, 3] 

correlate() 메서드의 mode를 'full'로 지정하면 풀패딩 방식을 이용한다.

In [6]:
correlate(x, w, mode='full')

array([ 6, 34, 51, 48, 57, 24, 25, 16, 39, 29, 13, 10])

#**Same Padding: 세임 패딩**

세임패딩은 아래와 같이 출력 배열의 길이가 원본 배열의 길이와 같아지도록   
원본 배열에 제로 패딩을 추가하는 방식이다.

    x = [0, 0, 2, 8, 3, 7, 1, 2, 0, 4, 5, 0]   
    w = [2, 1, 5, 3]   
       w = [2, 1, 5, 3]   
          w = [2, 1, 5, 3]   
             w = [2, 1, 5, 3]   
                w = [2, 1, 5, 3]   
                   w = [2, 1, 5, 3]   
                      w = [2, 1, 5, 3]   
                         w = [2, 1, 5, 3]   
                            w = [2, 1, 5, 3]   

correlate() 메서드의 mode를 'same'으로 지정한다.

In [7]:
correlate(x, w, mode='same')

array([34, 51, 48, 57, 24, 25, 16, 39, 29])

합성곱 신경망에서는 밸리드 패딩이나 풀패딩을 잘 사용하지는 않는다.   
대부분 세임 패딩(Same padding)을 사용한다.

#**Stride: 스트라이드**

스트라이드는 미끄러지는 배열의 간격을 말한다.   
스트라이드를 1로 지정하면 앞에서 살펴본 그림과 같이 한 칸씩 미끄러져 연산된다.   
보통 합성곱 신경망을 구현할 때에는 스트라이드를 1로 지정한다.

#**2차원 배열 합성곱**

    x = [[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]
    w = [[2, 0], 
         [0, 0]]

1. 원본 배열의 왼쪽 모서리 끝에 미끄러지는 배열을 맞춘 다음 합성곱을 수행한다.   
2. 미끄러지는 배열을 오른쪽으로 1칸 옮겨 합성곱을 수행한다. 
3. 미끄러지는 배열이 원본 배열의 끝에 도달하면 아래로 한 칸 내려 다시 왼쪽부터 합성곱을 수행한다.

위 그림의 경우 총 4번의 연산을 수행한다.    
correlate2d() 메서드를 사용해 합성곱을 계산해보자.

In [8]:
from scipy.signal.signaltools import correlate2d

x = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])

w = np.array([[2, 0], [0, 0]])

correlate2d(x, w, mode='valid')

array([[ 2,  4],
       [ 8, 10]])

In [10]:
correlate2d(x, w, mode='same')  # 세임 패딩을 사용했기 때문에 원본 배열과 같은 크기의 배열이 출력된다.

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]])

#**Convolution using Tensorflow**

텐서플로에서 2차원 합성곱을 수행하는 메서드는 conv2d()이다.   
conv2d() 메서드는 입력으로 4차원 배열(입력 이미지 높이, 너비, + 더 많은 차원)을 기대한다.

|--|샘플|너비|--|
|:--:|:--:|:--:|:--:|
|ㅣ|3|2|7|
|높이|1|10|6|
|ㅣ|4|2|5|
<br>

|--|샘플|너비|--|
|:--:|:--:|:--:|:--:|
|ㅣ|3|2|7|
|높이|1|10|6|
|ㅣ|4|2|5|

4차원 입력 배열은 순서대로 (배치, 샘플 높이, 샘플 너비, 컬러채널)이다.   
위의 입력을 4차원 배열로 표현하면 (2, 3, 3, 3)이다.

입력과 곱해지는 가중치 역시도 4개의 차원으로 구성된다.   
(가중치 높이, 너비, 채널, 가중치 개수) 이다.

||가중치|너비|
|:--:|:--:|:--:|
|높|2|0|
|이|0|0|

<br>

||가중치|너비|
|:--:|:--:|:--:|
|높|7|0|
|이|0|0|

<br>

||가중치|너비|
|:--:|:--:|:--:|
|높|0|0|
|이|0|5|

위 가중치의 경우 (2, 2, 3, 3)으로 표현한다.   

합성곱을 수행하면 (입력의 배치, 높이, 너비, 가중치 개수)의 출력이 나온다.    

###**2차원 배열을 4차원 배열로 바꿔 합성곱 수행**

앞에서 사용한 x, w 배열을 넘파이의 reshape() 메서드로 4차원 배열로 바꾸고,   
텐서플로는 실수형 입력을 기대하므로 넘파이의 astype() 메서드로 자료형을 실수형으로 바꾼다.   
배치와 컬러채널은 1로 지정한다.

In [11]:
import tensorflow as tf

x_4d = x.astype(np.float).reshape(1, 3, 3, 1)
w_4d = w.astype(np.float).reshape(2, 2, 1, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [12]:
c_out = tf.nn.conv2d(x_4d, w_4d, strides=1, padding='SAME')   # 스트라이드는 1, 세임 패딩으로 지정
'''
  conv2d()는 결과값으로 Tensor 객체를 리턴한다.
  Tensor 객체는 다차원의 배열로, Tensor 객체의 numpy() 메서드를 사용해
  넘파이 배열로 변환할 수 있다.
  여기서는 편의를 위해 배치 차원과 컬러 차원을 제거하고 (3, 3)크기로 변환하도록 하자.
'''

'\n  conv2d()는 결과값으로 Tensor 객체를 리턴한다.\n  Tensor 객체는 다차원의 배열로, Tensor 객체의 numpy() 메서드를 사용해\n  넘파이 배열로 변환할 수 있다.\n  여기서는 편의를 위해 배치 차원과 컬러 차원을 제거하고 (3, 3)크기로 변환하도록 하자.\n'

In [13]:
c_out.numpy().reshape(3, 3)

array([[ 2.,  4.,  6.],
       [ 8., 10., 12.],
       [14., 16., 18.]])

##**Filter/Kernel: 필터 혹은 커널**

합성곱의 가중치를 종종 '필터' 혹은 '커널'로 부른다.   
텐서플로의 케라스는 합성곱의 가중치를 '커널'이라고 부른다.   
여기서는 합성곱 필터 1개를 지칭할 때에는 '커널',
전체 필터를 지칭할 때에는 '가중치'라는 용어를 사용할 것이다.